### Conway's Game of Life - Unlimited Edition
##### Codewars | 4 kyu | 52423db9add6f6fc39000354

Given a 2D array and a number of generations, compute n timesteps of Conway's Game of Life. The rules of the game are:

1. Any live cell with fewer than two live neighbours dies
2. Any live cell with more than three live neighbours dies
3. Any cell with two or three live neighbours lives to the next generation
4. Any dead cell with exactly three live neighbours becomes a live cell 

A cell's neighborhood is the 8 cellls immediately around it. The universe is infinite, however the returned value should be a 2D array cropped around all living cells. In the event of no living cells, then return [[]].

##### Brainstorming
- Can convert the 2D array to a numpy array and use convolve to determine which cells should live onto the next generation. Can then use a dictionary to map the results to an alive or dead cell.
- Can trim after every step versus only at the end to avoid needless additional convolutions. Trimming can be done by multiplying mx1 to figure out top and bottom trimming and a transposed version of matrix with nx1 to figure out side trimming. Stop mul once you hit a nonzero result
- After the above two steps, can repeat in a loop for the required iterations and convert back into a 2D list

In [4]:
import subprocess
import sys

def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_and_import('scipy')

In [59]:
from collections import defaultdict
import numpy as np   
from scipy import signal

def get_generation(cells : list[list[int]], generations : int) -> list[list[int]]:
    
    # Filter to isolate cases 
    f = np.array([
        [1, 1, 1],
        [1, 10, 1],
        [1, 1, 1]
    ])
    
    # Converts filter outputs to a live or unalive cell
    d = defaultdict(lambda: 0)
    d[3], d[12], d[13] = 1, 1, 1
    
    # Conversion to numpy array makes it easier
    c = np.array(cells) 

    for _ in range(generations):
        
        # pads whole array with 0s
        c_pad = np.pad(c, 1, mode = 'constant')
        
        # Filter to isolate the different cases
        c_conv = signal.convolve2d(c_pad, f, boundary = 'symm', mode = 'same')
        
        # Converts to live/unalive cells format
        c_dict = np.vectorize(d.get)(c_conv)
        c_dict[c_dict == None] = 0

        # Trims extra columns and rows per iteration versus all at the end
        cm = np.where(c_dict != 0)
        c = c_dict[min(cm[0]) : max(cm[0]) + 1, min(cm[1]) : max(cm[1]) + 1].astype(int)

    return c.tolist()